In [1]:
## Installation et Import des packages necessaires (pandas, Nltk)
!pip3 install --user -U numpy
!pip3 install --user -U nltk
!pip3 install pandas

## Import de packages necessaires pour la suite
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import pandas as pd 
import xml.etree.ElementTree as et 
from nltk.corpus import wordnet
import nltk
import difflib
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords 
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 
lem = WordNetLemmatizer() 
nltk.download('sentiwordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')

Requirement already up-to-date: numpy in /home/deptinfo/.local/lib/python3.8/site-packages (1.19.4)
Requirement already up-to-date: nltk in /home/deptinfo/.local/lib/python3.8/site-packages (3.5)


[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/deptinfo/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/deptinfo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/deptinfo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/deptinfo/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/deptinfo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/deptinfo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#Extraction des données
#suppression des données inutiles (ne comportant pas d'aspect terms)
#Suppresion des polarité conflictuelle
def xmltodata(xml):
    df = et.parse(xml)
    data  = df.getroot()

    texte = []
    for texts in data:
        term = []
        for text in texts:
            if text.tag == "text" :
                term.append(text.text)
            elif text.tag == "aspectTerms":
                for at in text :
                    for pol,val in at.attrib.items() :
                        if pol == 'polarity' :
                            if val != "conflict":
                                term.append(at.attrib)
        if len(term)>1:
            texte.append(term)
    return texte

In [3]:
#Récuperation des mots negatifs et positifs de la BD "Sentiment Lexicon " sur Kaggle
df1 = pd.read_fwf('negative_words_en.txt', header=None, index=False)
negatif = df1.values.tolist()
df2 = pd.read_fwf('positive_words_en.txt', header=None, index=False)
positif = df2.values.tolist()

In [4]:
#On simplifie le dictionnaire de chaque commentaire en transformant la val
#de polarity avec des chiffres
#negatif = -1 ; neutre = 0 ; positif = 1
def simple_polarity(data):
    for i in range(len(data)):
        for j in range(len(data[i])-1):
            if data[i][j+1]['polarity'] == "negative":
                data[i][j+1]['polarity'] = -1
            elif data[i][j+1]['polarity'] == "neutral":
                data[i][j+1]['polarity'] = 0
            elif data[i][j+1]['polarity'] == "positive":
                data[i][j+1]['polarity'] = 1
    return data

In [5]:
#Retire les stop_word
def no_stopword(word):
    stop_words = set(stopwords.words('english'))
    token = nltk.word_tokenize((word.lower()))
    tagged = nltk.pos_tag(token)
    filtered_sentence = []
    for w in token:  
        if w not in stop_words:  
            filtered_sentence.append(w) 
    return filtered_sentence 

In [6]:
#Segmentation de la phrase pour chaque aspect terme en fonction de la ponctuation ou de sa position dans la phrase
def flag_text(texte):
    text = texte[0]
    aspects_terms = []
    for i in range(len(texte)-1):
        aspects_terms.append(texte[i+1]['term'])
    sentences = re.split(r' *[\.\?!,][\'"\)\]]* *', text)
    if "" in sentences :
        sentences.remove("")    
    if len(sentences) > 1 :
        dict_terms_sentences = {curr_aspect : [curr_sentence for curr_sentence in sentences 
                                                                if curr_aspect.lower() in curr_sentence.lower()] 
                                                                for curr_aspect in aspects_terms}
    elif len(aspects_terms) > 1 : 
        for term in aspects_terms :
            textsplit = []
            split = text.split(term)           
            if len(textsplit) < 1:
                textsplit.append(split[0] + " " + term)
            else :
                textsplit.append(split[1]+ " " + term)        
        dict_terms_sentences = {curr_aspect : [curr_sentence for curr_sentence in textsplit
                                                                if curr_aspect.lower() in curr_sentence.lower()]
                                                                for curr_aspect in aspects_terms}
    else :     
        dict_terms_sentences = { aspects_terms[0] : [text]}
    
    return dict_terms_sentences

In [49]:
#Récupération des Keywords qui sont susceptibles d'avoir un lien
#Avec le sentiment
#EN prenant en compte la possible eventualité que nous ne trouvons pas de keywords (donc mot neutre)
def keywords_aspect(dic):
    keyWord = []
    neg = []
    for words in dic:
        text = dic[words]
        if len(dic[words])>0:
            aspect = words
            token = nltk.word_tokenize((text[0].lower()))
            tagged = nltk.pos_tag(token)
            aspectWord = []
            for i in range(len(tagged)):
                if tagged[i][1] in ['RP','JJ','RB','NN']:
                    aspectWord.append(tagged[i])
            if aspectWord == [] :
                aspectWord.append("")
            keyWord.append(aspectWord)
        else :
            keyWord.append([""])
    return keyWord

In [50]:
#Récupération des Keywords qui sont susceptibles d'avoir un lien
#Avec le sentiment de l'aspect en prenant en compte les "not et n't avant le mot"
#C'est la version utilisé
def keywords_aspect_not(dic):
    keyWord = []
    neg = []
    for words in dic:
        text = dic[words]
        if len(dic[words])>0:
            aspect = words
            token = nltk.word_tokenize((text[0].lower()))
            tagged = nltk.pos_tag(token)
            aspectWord = []
            for i in range(len(tagged)):
                if tagged[i][1] in ['RP','JJ','RB','NN']:
                    if i == 0 :
                        aspectWord.append(["p",tagged[i]])
                    if i > 0:
                        if tagged[i-1][0] == "not" or tagged[i-1][0] =="n't":
                            aspectWord.append(["n",tagged[i]])
                        else :
                            aspectWord.append(["p",tagged[i]])
            if aspectWord == [] :
                aspectWord.append(["p",""])
            keyWord.append(aspectWord)
        else :
            keyWord.append([["p",""]])
    return keyWord

In [9]:
#Fonction qui applique keywords_aspect() a toute la data
def keywords_data_not(data):
    keywords = []
    for i in range(len(data)):
        keywords.append(keywords_aspect_not(flag_text(data[i])))
    return keywords 

In [10]:
#Fonction qui applique keywords_aspect_not() a toute la data
#C'est la version utilisé
def keywords_data(data):
    keywords = []
    for i in range(len(data)):
        keywords.append(keywords_aspect(flag_text(data[i])))
    return keywords 

In [11]:
#Fonction qui va calculer la polarité en comparant les keywords avec 
#les données positifs/negatifs récupéré sur kaggle
def com_polarity(data):
    datalist = []
    for com in data :
        comlist =  []
        for aspect in com:
            aspectlist = []
            for tagged in aspect :
                if len(tagged) != 0 :
                    word = lem.lemmatize(tagged[0])
                else :
                    word = " "
                nbneg = 0
                nbpos = 0
                for mot_neg in negatif:
                    if word == mot_neg[0]:
                        nbneg += 1
                for mot_pos in positif:
                    if word == mot_pos[0]:
                        nbpos += 1       
                if nbneg>nbpos :
                    aspectlist.append(-1)
                elif nbneg<nbpos:
                    aspectlist.append(1)
                elif nbneg == nbpos:
                    aspectlist.append(0)
            if len(aspectlist) < 1 :
                aspectlist.append(0)
            somme = sum(aspectlist)
            if somme > 0 :
                comlist.append(1)
            elif somme < 0 :
                comlist.append(-1)
            else :               
                comlist.append(0)
                
        datalist.append(comlist)
    return datalist 

In [12]:
#Cette version prends en compte les "not" et "n't"
#Fonction qui va calculer la polarité en comparant les keywords avec 
#les données positifs/negatifs récupéré sur kaggle
#C'est la version utilisé
def com_polarity_not(data):
    datalist = []
    for com in data :
        #print(com)
        comlist =  []
        for aspect in com:
            #print(aspect)
            aspectlist = []
            for tagged in aspect :
                if len(tagged[1]) != 0 :
                    word = lem.lemmatize(tagged[1][0])
                else :
                    word = " "
                nbneg = 0
                nbpos = 0
                if tagged[0] == 'p' :
                    for mot_neg in negatif:
                        if word == mot_neg[0]:
                            nbneg += 1
                    for mot_pos in positif:
                        if word == mot_pos[0]:
                            nbpos += 1  
                elif tagged[0] == 'n':
                    for mot_neg in negatif:
                        if word == mot_neg[0]:
                            nbpos += 1
                    for mot_pos in positif:
                        if word == mot_pos[0]:
                            nbneg += 1 
                    
                if nbneg>nbpos :
                    aspectlist.append(-1)
                elif nbneg<nbpos:
                    aspectlist.append(1)
                elif nbneg == nbpos:
                    aspectlist.append(0)
            if len(aspectlist) < 1 :
                aspectlist.append(0)
            somme = sum(aspectlist)
            if somme > 0 :
                comlist.append(1)
            elif somme < 0 :
                comlist.append(-1)
            else :               
                comlist.append(0)
                
        datalist.append(comlist)
    return datalist 

In [13]:
#Enumere le nombre de positif, neutre et negatif par phrase (reponse)
#Non utilisé
def numaspect(data):
    poldata = []
    for j in range(len(data)):
        neg = 0
        pos = 0
        neu = 0
        for i in range(len(data[j])-1):
            if data[j][i+1]['polarity'] == -1:
                neg += 1
            elif data[j][i+1]['polarity'] == 1:
                pos += 1
            else :
                neu += 1
        poldata.append([neg,pos,neu])            
    return poldata      

In [14]:
#enumere la polarité des terms (reponses)
def pol_doc_aspect(data):
    res = []
    for com in data:
        polaspect = []
        for i in range(len(com)-1):
            polaspect.append(com[i+1]['polarity'])
        res.append(polaspect)
    return res

In [98]:
#Ce sont les fcts qui vont appliaquer toutes les fct précédente a la data
def preparation_data(fichier) :
    data_train = xmltodata(fichier)
    data_train_simple = simple_polarity(data_train)
    data_keywords = keywords_data_not(data_train_simple)
    pol_asp_data = com_polarity_not(data_keywords)
    
    return pol_asp_data

def preparation_rep(fichier):
    data_test= xmltodata(fichier)
    data_simple = simple_polarity(data_test)
    #data_test_pol = numaspect(data_test)
    data_val = pol_doc_aspect(data_simple)
    return data_val

# APPLICATION DES FONCTIONS SUR LES DATAS

In [99]:
data_resto_train = preparation_data("Restaurants_Train.xml")
res_resto_train = preparation_rep("Restaurants_Train.xml")

data_resto_test = preparation_data("Restaurants_Test_Gold.xml")
res_resto_test = preparation_rep("Restaurants_Test_Gold.xml")

In [91]:
data_laptop_train = preparation_data("Laptop_Train.xml")
res_laptop_train = preparation_rep("Laptop_Train.xml")

data_laptop_test = preparation_data("Laptop_Test_Gold.xml")
res_laptop_test = preparation_rep("Laptop_Test_Gold.xml")

In [134]:
data_laptop_nolabel = preparation_data("Laptop_Test_NoLabels.xml")
data_Restaurants_nolabel = preparation_data("Restaurants_Test_NoLabels.xml")

# EVALUTATION

In [126]:
def evaluation(data_res,test_data) :   
    repjuste = 0
    repfaus = 0
    totalinspectid = 0
    totalinspectrep = 0
    for i in range(len(data_res)):
        for k in range(len(data_res[i])):
            totalinspectrep += 1
        for j in range(len(test_data[i])):
            if test_data[i][j] == data_res[i][j]:
                repjuste += 1
                totalinspectid +=1
            elif test_data[i][j] != data_res[i][j]:
                repfaus +=1
                totalinspectid +=1
                
                
    precision = repjuste/totalinspectid
    rappel = repjuste/totalinspectrep
    print("precision : " ,precision)
    print("rappel : ",rappel)
    print("Resultat final : ", 2*((precision*rappel)/(precision+rappel)))

In [127]:
print("Evaluation sur le dataset : Restaurants_Train.xml")
evaluation(res_resto_train,data_resto_train)

precision :  0.5326421966937518
rappel :  0.5277623542476402
Resultat final :  0.5301910472737414


In [128]:
print("Evaluation sur le dataset : Restaurants_Test_Gold.xml")
evaluation(res_resto_test,data_resto_test)

precision :  0.6979166666666666
rappel :  0.6979166666666666
Resultat final :  0.6979166666666666


In [129]:
print("Evaluation sur le dataset : Laptop_Train.xml")
evaluation(res_laptop_test,data_laptop_test)

precision :  0.4897959183673469
rappel :  0.4897959183673469
Resultat final :  0.4897959183673469


In [130]:
print("Evaluation sur le dataset : Laptop_Test_Gold.xml")
evaluation(res_laptop_train,data_laptop_train)

precision :  0.48610498456109397
rappel :  0.4764375270211846
Resultat final :  0.4812227074235808
